In [7]:
!pip install pyngrok streamlit PyMuPDF anthropic python-dotenv

In [20]:
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("")

In [25]:
%%writefile app.py
import os
from dotenv import load_dotenv
import fitz
import streamlit as st

from anthropic import Anthropic
client = Anthropic(api_key="")


def extract_text_from_pdf(file):
    doc = fitz.open(stream=file.read(), filetype="pdf")
    return "\n".join([page.get_text() for page in doc])

def summarize_with_claude(text):
    prompt = f"""You are a helpful academic assistant. Summarize the following paper:
    - What is it about?
    - What are the key contributions?
    - What methods were used?
    - What are the main findings?

    {text[:12000]}"""  # Truncate if needed

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1000,
        temperature=0.3,
        system="Summarize academic papers clearly.",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.content[0].text

st.title("📘 Claude-Powered Paper Summarizer")
uploaded_file = st.file_uploader("Upload a research paper (PDF)", type=["pdf"])

if uploaded_file:
    with st.spinner("Extracting text..."):
        paper_text = extract_text_from_pdf(uploaded_file)
    st.success("Text extracted!")

    if st.button("🧠 Summarize with Claude"):
        with st.spinner("Talking to Claude..."):
            summary = summarize_with_claude(paper_text)
        st.subheader("📝 Summary")
        st.write(summary)

    st.subheader("🧠 Ask a Question About This Paper")

    user_question = st.text_input("Enter your question here")

    if user_question:
        with st.spinner("Getting an answer..."):
            qa_prompt = f"""You are an expert academic assistant.
Answer the following question based on the content of this research paper:

Paper Content:
{paper_text[:12000]}

Question: {user_question}

Answer:"""

            answer = client.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=500,
                temperature=0.2,
                system="You answer academic questions clearly and concisely.",
                messages=[{"role": "user", "content": qa_prompt}]
            )
            st.markdown(f"**Answer:**\n\n{answer.content[0].text}")

def generate_glossary(text):
    prompt = f"""You are an AI assistant trained to simplify academic language.

From the following paper, extract the 5–10 most important technical terms or concepts. For each term, write a short definition in simple, clear language.

Output in this format:

Term: Definition

Text:
{text[:12000]}
"""

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=800,
        temperature=0.3,
        system="You generate glossaries from technical papers.",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text



st.subheader("📘 Glossary of Key Terms")
if st.button("📖 Generate Glossary"):
    with st.spinner("Extracting glossary terms..."):
        glossary = generate_glossary(paper_text)
    st.text(glossary)

def explain_like_12(text):
    prompt = f"""You are an educational AI assistant.

Explain the following research paper as if you're speaking to a curious 12-year-old. Avoid jargon and keep it simple, like you're telling a story.

Text:
{text[:12000]}
"""

    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=800,
        temperature=0.4,
        system="You explain complex topics in a child-friendly way.",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text

st.subheader("🧒 Explain Like I'm 12")
if st.button("🧠 Simplify This Paper"):
    with st.spinner("Explaining in kid-friendly language..."):
        simple_explanation = explain_like_12(paper_text)
    st.write(simple_explanation)



Overwriting app.py


In [26]:
# Start Streamlit in background
!streamlit run app.py &>/content/logs.txt &

# Now create the tunnel using HTTP
public_url = ngrok.connect("8501", "http")  # explicitly specify HTTP type
print("🌐 Streamlit Public URL:", public_url)


🌐 Streamlit Public URL: NgrokTunnel: "https://170b-34-82-124-236.ngrok-free.app" -> "http://localhost:8501"
